In [1]:
import pandas as pd
import os

# File paths 
txt_file_path = 'tu.series.txt' 
excel_file_path = 'tu_a1_data.xlsx'
output_excel_path = 'updated_tu_a1_data.xlsx'
csv_output_folder = 'csv_outputs'  # Folder to store the CSV files

# Step 1: Load the series data from the .txt file
def load_series_info(txt_file):
    series_data = {}
    print(f"Loading series data from {txt_file}...")
    with open(txt_file, 'r', encoding='utf-8') as file:
        # Skip the header row
        header = file.readline()
        for line in file:
            parts = line.strip().split('\t')  # Tab-delimited
            if len(parts) > 1:  # Ensure the line has enough columns
                series_id = parts[0].strip()  # The first column is series_id
                series_title = parts[-6].strip()  # The "series_title" column (2nd to last before footnotes)
                series_data[series_id] = series_title
    print(f"Loaded {len(series_data)} series from the text file.")
    return series_data

# Step 2: Update the Excel file with split series_title columns
def update_excel_with_split_titles(excel_file, txt_file, output_file):
    # Load series data
    series_data = load_series_info(txt_file)

    # Open the Excel file
    print(f"Processing Excel file: {excel_file}")
    excel_data = pd.ExcelFile(excel_file, engine='openpyxl')
    updated_sheets = {}

    for sheet_name in excel_data.sheet_names:
        print(f"Processing sheet: {sheet_name}")
        if sheet_name.lower() == "readme":
            # Copy the readme sheet without modifications
            readme_df = pd.read_excel(excel_data, sheet_name=sheet_name)
            updated_sheets[sheet_name] = readme_df
        else:
            # Load the sheet and update it
            df = pd.read_excel(excel_data, sheet_name=sheet_name)

            if 'seriesid' in df.columns:
                # Map series_title from the text file
                df['series_title'] = df['seriesid'].map(series_data)

                # Use a regex that works for both formats
                split_data = df["series_title"].str.extract(
                    r"^(.*?) - (.*?), (.*?)(?:, (Men|Women))?$"
                )
                df["Series"] = split_data[0]  # Series name
                df["Activity"] = split_data[1]  # Activity name
                df["Age Group"] = split_data[2]  # Age group
                df["Gender"] = split_data[3].fillna("MenAndWomen")  # Gender (if missing, fill with indicated string)

                # Drop the original series_title column (optional)
                df.drop(columns=["series_title"], inplace=True)

            updated_sheets[sheet_name] = df
    
    # Save the updated Excel file
    print(f"Saving updated Excel file to: {output_file}")
    with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
        for sheet_name, sheet_df in updated_sheets.items():
            sheet_df.to_excel(writer, sheet_name=sheet_name, index=False)

    print("Processing complete.")

    # Export updated sheets to separate CSV files (excluding readme sheet)
    export_sheets_to_csv(output_file, csv_output_folder)

# Step 3: Export each sheet to a separate CSV file (skip readme sheet)
def export_sheets_to_csv(updated_excel_file, output_folder):
    print(f"Exporting sheets from {updated_excel_file} to CSV files in folder {output_folder}...")
    
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Load the updated Excel file
    excel_data = pd.ExcelFile(updated_excel_file, engine='openpyxl')

    for sheet_name in excel_data.sheet_names:
        # Skip the "readme" sheet
        if sheet_name.lower() == "readme":
            print(f"Skipping sheet: {sheet_name} for CSV export.")
            continue  # Skip the readme sheet

        print(f"Exporting sheet: {sheet_name}")
        df = pd.read_excel(excel_data, sheet_name=sheet_name)
        csv_file_path = os.path.join(output_folder, f"{sheet_name}.csv")
        df.to_csv(csv_file_path, index=False)
        print(f"Saved: {csv_file_path}")

    print("CSV export complete.")

# Run the update
update_excel_with_split_titles(excel_file_path, txt_file_path, output_excel_path)

ModuleNotFoundError: No module named 'pandas'